d-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px; height: 163px">
</div>

# Databricks Delta Optimizations and Best Practices

Databricks&reg; Delta has nifty optimizations to speed up your queries.

## In this lesson you:
* Optimize a Databricks Delta data pipeline backed by online shopping data
* Learn about best practices to apply to data pipelines

## Audience
* Primary Audience: Data Engineers 
* Secondary Audience: Data Analysts and Data Scientists

## Prerequisites
* Web browser: **Chrome**
* A cluster configured with **8 cores** and **DBR 6.2**
* Suggested Courses from <a href="https://academy.databricks.com/" target="_blank">Databricks Academy</a>:
  - ETL Part 1
  - Spark-SQL

## Datasets Used
* Online retail datasets from
`/mnt/training/online_retail`

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Classroom-Setup

For each lesson to execute correctly, please make sure to run the **`Classroom-Setup`** cell at the<br/>
start of each lesson (see the next cell) and the **`Classroom-Cleanup`** cell at the end of each lesson.

In [4]:
%run "./Includes/Classroom-Setup"

<iframe  
src="//fast.wistia.net/embed/iframe/86vcy1wkdy?videoFoam=true"
style="border:1px solid #1cb1c2;"
allowtransparency="true" scrolling="no" class="wistia_embed"
name="wistia_embed" allowfullscreen mozallowfullscreen webkitallowfullscreen
oallowfullscreen msallowfullscreen width="640" height="360" ></iframe>
<div>
<a target="_blank" href="https://fast.wistia.net/embed/iframe/86vcy1wkdy?seo=false">
  <img alt="Opens in new tab" src="https://files.training.databricks.com/static/images/external-link-icon-16x16.png"/>&nbsp;Watch full-screen.</a>
</div>

Set up relevant paths.

In [7]:
deltaDataPath = workingDir + "/customer-data-delta/"

## SMALL FILE PROBLEM

Historical and new data is often written in very small files and directories. 

This data may be spread across a data center or even across the world (that is, not co-located).

The result is that a query on this data may be very slow due to
* network latency 
* volume of file metatadata 

The solution is to compact many small files into one larger file.
Databricks Delta has a mechanism for compacting small files.

-sandbox

Use Amazon S3's file browser to see many small files.

<img alt="Caution" title="Caution" style="vertical-align: text-bottom; position: relative; height:1.3em; top:0.0em" src="https://files.training.databricks.com/static/images/icon-warning.svg"/> S3's file browser is available ONLY in the AWS console (not in Databricks)

<img src="https://files.training.databricks.com/images/eLearning/Delta/amazon-small-file.png" style="border: 1px solid #aaa; border-radius: 10px 10px 10px 10px"/></div>

### OPTIMIZE
Databricks Delta supports the `OPTIMIZE` operation, which performs file compaction.

Small files are compacted together into new larger files up to 1GB.

`OPTIMIZE` does not do any kind of file clean up, so, at this point the number of files increases!

The 1GB size was determined by the Databricks optimization team as a trade-off between query speed and run-time performance when running Optimize.

`OPTIMIZE` is not run automatically because you must collect many small files first.

* Run `OPTIMIZE` more often if you want better end-user query performance 
* Since `OPTIMIZE` is a time consuming step, run it less often if you want to optimize cost of compute hours
* To start with, run `OPTIMIZE` on a daily basis (preferably at night when spot prices are low), and determine the right frequency for your particular business case
* In the end, the frequency at which you run `OPTIMIZE` is a business decision

The easiest way to see what `OPTIMIZE` does is to perform a simple `count(*)` query before and after and compare the timing!

### Repopulate Data Set

You may have deleted the files created in previous lessons.

We re-create them for you.

In [12]:

from pyspark.sql.functions import expr, col, from_unixtime, to_date
jsonSchema = "action string, time long"
streamingEventPath = "/mnt/training/structured-streaming/events/"
deltaIotPath  = workingDir + "/iot-pipeline/"

rawDataDF = (spark
  .read 
  .schema(jsonSchema)
  .json(streamingEventPath) 
  .withColumn("date", to_date(from_unixtime(col("time").cast("Long"),"yyyy-MM-dd")))
  .withColumn("deviceId", expr("cast(rand(5) * 100 as int)"))
  .repartition(200)
  .write
  .mode("overwrite")
  .format("delta")
  .partitionBy("date")
  .save(deltaIotPath)
)

Take a look at a subdirectory of `deltaIotPath`.

Notice, hundreds of files like `../delta/iot-pipeline/date=xxxx-xx-xx/part-xxxx.snappy.parquet`.

In [14]:
try:
  print(dbutils.fs.ls(dbutils.fs.ls(deltaIotPath)[1].path))
except:
  print("There are no files in deltaIotPath")

[FileInfo(path='dbfs:/user/tbresee@umich.edu/delta/delta_06_optimization_psp/iot-pipeline/date=2016-07-26/part-00000-f8c96f10-871f-40f7-8833-f8c1f1106563.c000.snappy.parquet', name='part-00000-f8c96f10-871f-40f7-8833-f8c1f1106563.c000.snappy.parquet', size=2474), FileInfo(path='dbfs:/user/tbresee@umich.edu/delta/delta_06_optimization_psp/iot-pipeline/date=2016-07-26/part-00001-6a4c9fe1-340a-464f-834c-afa774972181.c000.snappy.parquet', name='part-00001-6a4c9fe1-340a-464f-834c-afa774972181.c000.snappy.parquet', size=2446), FileInfo(path='dbfs:/user/tbresee@umich.edu/delta/delta_06_optimization_psp/iot-pipeline/date=2016-07-26/part-00002-ca783405-4f82-487b-bb9a-ac6b94506adb.c000.snappy.parquet', name='part-00002-ca783405-4f82-487b-bb9a-ac6b94506adb.c000.snappy.parquet', size=2494), FileInfo(path='dbfs:/user/tbresee@umich.edu/delta/delta_06_optimization_psp/iot-pipeline/date=2016-07-26/part-00003-b2319cee-3288-4d61-92f7-51896bbb4ab3.c000.snappy.parquet', name='part-00003-b2319cee-3288-4d61-92f7-51896bbb4ab3.c000.snappy.parquet', size=2429), FileInfo(path='dbfs:/user/tbresee@umich.edu/delta/delta_06_optimization_psp/iot-pipeline/date=2016-07-26/part-00004-96b059fd-f599-497b-8481-09004b8ec529.c000.snappy.parquet', name='part-00004-96b059fd-f599-497b-8481-09004b8ec529.c000.snappy.parquet', size=2425), FileInfo(path='dbfs:/user/tbresee@umich.edu/delta/delta_06_optimization_psp/iot-pipeline/date=2016-07-26/part-00005-51e5fa77-6b83-4b8c-8231-950b772d54e6.c000.snappy.parquet', name='part-00005-51e5fa77-6b83-4b8c-8231-950b772d54e6.c000.snappy.parquet', size=2470), FileInfo(path='dbfs:/user/tbresee@umich.edu/delta/delta_06_optimization_psp/iot-pipeline/date=2016-07-26/part-00006-e64cb847-2809-4004-8d8d-90f6493cc4ac.c000.snappy.parquet', name='part-00006-e64cb847-2809-4004-8d8d-90f6493cc4ac.c000.snappy.parquet', size=2472), FileInfo(path='dbfs:/user/tbresee@umich.edu/delta/delta_06_optimization_psp/iot-pipeline/date=2016-07-26/part-00007-fcb55f48-7a3d-42b6-ba40-3d295e2478f5.c000.snappy.parquet', name='part-00007-fcb55f48-7a3d-42b6-ba40-3d295e2478f5.c000.snappy.parquet', size=2546), FileInfo(path='dbfs:/user/tbresee@umich.edu/delta/delta_06_optimization_psp/iot-pipeline/date=2016-07-26/part-00008-5248c2d4-8615-414b-a867-77bc194facac.c000.snappy.parquet', name='part-00008-5248c2d4-8615-414b-a867-77bc194facac.c000.snappy.parquet', size=2465), FileInfo(path='dbfs:/user/tbresee@umich.edu/delta/delta_06_optimization_psp/iot-pipeline/date=2016-07-26/part-00009-7ea9782e-e024-4f02-a8e3-08b135cad717.c000.snappy.parquet', name='part-00009-7ea9782e-e024-4f02-a8e3-08b135cad717.c000.snappy.parquet', size=2468), FileInfo(path='dbfs:/user/tbresee@umich.edu/delta/delta_06_optimization_psp/iot-pipeline/date=2016-07-26/part-00010-63a5202a-5339-4f64-b794-b54a9660f683.c000.snappy.parquet', name='part-00010-63a5202a-5339-4f64-b794-b54a9660f683.c000.snappy.parquet', size=2500), FileInfo(path='dbfs:/user/tbresee@umich.edu/delta/delta_06_optimization_psp/iot-pipeline/date=2016-07-26/part-00011-51583f28-1254-4f85-af2f-aeaf841be86d.c000.snappy.parquet', name='part-00011-51583f28-1254-4f85-af2f-aeaf841be86d.c000.snappy.parquet', size=2494), FileInfo(path='dbfs:/user/tbresee@umich.edu/delta/delta_06_optimization_psp/iot-pipeline/date=2016-07-26/part-00012-4015c2a1-ca78-4358-9396-0c414859a23d.c000.snappy.parquet', name='part-00012-4015c2a1-ca78-4358-9396-0c414859a23d.c000.snappy.parquet', size=2543), FileInfo(path='dbfs:/user/tbresee@umich.edu/delta/delta_06_optimization_psp/iot-pipeline/date=2016-07-26/part-00013-7403dccb-094b-4f98-b6f2-fdd7c66421ff.c000.snappy.parquet', name='part-00013-7403dccb-094b-4f98-b6f2-fdd7c66421ff.c000.snappy.parquet', size=2492), FileInfo(path='dbfs:/user/tbresee@umich.edu/delta/delta_06_optimization_psp/iot-pipeline/date=2016-07-26/part-00014-297d728d-26e4-463f-a54d-f6456ec97b4b.c000.snappy.parquet', name='part-00014-297d728d-26e4-463f-a54d-f6456ec97b4b.c000.snappy.parquet', size=2487), FileInfo(path='dbfs:/user/tbresee@umich.edu/delta/delt

Pick a `deviceId` then run the `SELECT` query. 

Notice it is very slow, due to the large number of small files.

In [16]:
devID = spark.sql("SELECT deviceId FROM delta.`{}` limit 1".format(deltaIotPath)).first()[0]
iotDF = spark.sql("SELECT * FROM delta.`{}` where deviceId={}".format(deltaIotPath, devID))
display(iotDF)

action,time,date,deviceId
Open,1469624003,2016-07-27,81
Open,1469604673,2016-07-27,81
Open,1469604031,2016-07-27,81
Open,1469637342,2016-07-27,81
Open,1469652479,2016-07-27,81
Open,1469582391,2016-07-27,81
Close,1469593791,2016-07-27,81
Open,1469584307,2016-07-27,81
Open,1469605564,2016-07-27,81
Open,1469609533,2016-07-27,81


-sandbox
### Partition Pruning, Data Skipping and ZORDER

Databricks Delta uses multiple mechanisms to speed up queries.

<b>Partition Pruning</b> is a performance optimization that speeds up queries by limiting the amount of data read.

If the WHERE clause filters on a table partitioned column, then only table partitions (sub-directories) that may have matching records are read.  

For example, we have a data set that is partitioned by `date`. 

A query using `WHERE date > 2018-06-01` would not access data that resides in partitions that correspond to dates prior to `2018-06-01`.  

<b>Data Skipping</b> is a performance optimization that aims at speeding up queries that contain filters (WHERE clauses). 

As new data is inserted into a Databricks Delta table, file-level min/max statistics are collected for all columns (including nested ones) of supported types. Then, when there’s a lookup query against the table, Databricks Delta first consults these statistics in order to determine which files can safely be skipped.

<b>ZOrdering</b> is a technique to colocate related information in the same set of files. 

ZOrdering maps multidimensional data to one dimension while preserving locality of the data points. 

Given a column that you want to perform ZORDER on, say `OrderColumn`, Delta
* Takes existing parquet files within a partition.
* Maps the rows within the parquet files according to `OrderColumn` using <a href="https://en.wikipedia.org/wiki/Z-order_curve" target="_blank">this algorithm</a>.
* In the case of only one column, the mapping above becomes a linear sort.
* Rewrites the sorted data into new parquet files.

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> You cannot use the table partition column also as a ZORDER column.

-sandbox
### ZORDER example
In the image below, table `Students` has 4 columns: 
* `gender` with 2 distinct values
* `Pass-Fail` with 2 distinct values
* `Class` with 4 distinct values  
* `Student` with many distinct values 

Suppose you wish to perform the following query:

```SELECT Name FROM Students WHERE gender = 'M' AND Pass_Fail = 'P' AND Class = 'Junior'```

```ORDER BY Gender, Pass_Fail```

The most effective way of performing that search is to order the data starting with the largest set, which is `Gender` in this case. 

If you're searching for `gender = 'M'`, then you don't even have to look at students with `gender = 'F'`. 

Note that this technique is only beneficial if all `gender = 'M'` values are co-located.


<div><img src="https://files.training.databricks.com/images/eLearning/Delta/zorder.png" style="height: 300px"/></div><br/>

-sandbox
### ZORDER usage

With Databricks Delta the notation is:

> `OPTIMIZE Students`<br>
`ZORDER BY Gender, Pass_Fail`

This will ensure all the data backing `Gender = 'M' ` is colocated, then data associated with `Pass_Fail = 'P' ` is colocated.

See References below for more details on the algorithms behind ZORDER.

Using ZORDER, you can order by multiple columns as a comma separated list; however, the effectiveness of locality drops.

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> In streaming, where incoming events are inherently ordered (more or less) by event time, use `ZORDER` to sort by a different column, say 'userID'.

In [20]:
spark.sql("""OPTIMIZE delta.`{}` 
             ZORDER by (deviceID)""".format(deltaIotPath))

Out[29]: DataFrame[path: string, metrics: struct<numFilesAdded:bigint,numFilesRemoved:bigint,filesAdded:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,filesRemoved:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,partitionsOptimized:bigint,zOrderStats:struct<strategyName:string,inputCubeFiles:struct<num:bigint,size:bigint>,inputOtherFiles:struct<num:bigint,size:bigint>,inputNumCubes:bigint,mergedFiles:struct<num:bigint,size:bigint>,mergedNumCubes:bigint>,numBatches:bigint>]

The performance of the following query should now be much faster than it was before.

In [22]:
deviceDF = spark.sql("SELECT * FROM delta.`{}` WHERE deviceId={}".format(deltaIotPath, devID))
display(deviceDF)

action,time,date,deviceId
Open,1469624003,2016-07-27,81
Open,1469604673,2016-07-27,81
Open,1469604031,2016-07-27,81
Open,1469637342,2016-07-27,81
Open,1469652479,2016-07-27,81
Open,1469582391,2016-07-27,81
Close,1469593791,2016-07-27,81
Open,1469584307,2016-07-27,81
Open,1469605564,2016-07-27,81
Open,1469609533,2016-07-27,81


-sandbox
## VACUUM

To save on storage costs you should occasionally clean up invalid files using the `VACUUM` command. 

Invalid files are small files compacted into a larger file with the `OPTIMIZE` command.

The  syntax of the `VACUUM` command is 
>`VACUUM name-of-table RETAIN number-of HOURS;`

The `number-of` parameter is the <b>retention interval</b>, specified in hours.

<img alt="Caution" title="Caution" style="vertical-align: text-bottom; position: relative; height:1.3em; top:0.0em" src="https://files.training.databricks.com/static/images/icon-warning.svg"/> Databricks does not recommend you set a retention interval shorter than seven days because old snapshots and uncommitted files can still be in use by concurrent readers or writers to the table.

The scenario here is:
0. User A starts a query off uncompacted files, then
0. User B invokes a `VACUUM` command, which deletes the uncompacted files
0. User A's query fails because the underlying files have disappeared

Invalid files can also result from updates/upserts/deletions.

More details are provided here: <a href="https://docs.databricks.com/delta/optimizations.html#garbage-collection" target="_blank"> Garbage Collection</a>.

Count the number of files before we vacuum.

In [24]:
try:
  print(len(dbutils.fs.ls(dbutils.fs.ls(deltaIotPath)[1].path)))
except:
  print("There are no files in deltaIotPath")

201

In the example below we set off an immediate `VACUUM` operation with an override of the retention check so that all files are cleaned up immediately.

You would not do not do this in production!

If you do not set the override of the retention check, you get a helpful message.

```
requirement failed: Are you sure you would like to vacuum files with such a low retention period? If you have
writers that are currently writing to this table, there is a risk that you may corrupt the
state of your Delta table.

If you are certain that there are no operations being performed on this table, such as
insert/upsert/delete/optimize, then you may turn off this check by setting:
spark.databricks.delta.retentionDurationCheck.enabled = False
```

In [26]:
try:
  spark.sql(""" VACUUM delta.`{}` RETAIN 0 HOURS """.format(deltaIotPath))
except Exception as err: 
  print(str(err).replace("\\n", "\n").replace("'", ""))

requirement failed: Are you sure you would like to vacuum files with such a low retention period? If you have
writers that are currently writing to this table, there is a risk that you may corrupt the
state of your Delta table.

If you are certain that there are no operations being performed on this table, such as
insert/upsert/delete/optimize, then you may turn off this check by setting:
spark.databricks.delta.retentionDurationCheck.enabled = false

If you are not sure, please use a value not less than "168 hours".

In [27]:
spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled", False)

spark.sql(" VACUUM delta.`{}` RETAIN 0 HOURS ".format(deltaIotPath))

Out[33]: DataFrame[path: string]

Notice how the directory looks vastly cleaned up!

In [29]:
try:
  print(len(dbutils.fs.ls(dbutils.fs.ls(deltaIotPath)[1].path)))
except:
  print("There are no files in deltaIotPath")

1

# LAB

-sandbox
## Step 1: Repopulate Data

If you've deleted the data under `deltaDataPath = workingDir + "/customer-data-delta/"` in previous lessons, that is okay.

If the path no longer exists or has been cleaned out, repopulate data, otherwise, do nothing.

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> We are chaining the `read` and `write` operations in one statement.

In [32]:
inputPath = "/mnt/training/online_retail/data-001/data.csv"
inputSchema = "InvoiceNo STRING, StockCode STRING, Description STRING, Quantity INT, InvoiceDate STRING, UnitPrice DOUBLE, CustomerID INT, Country STRING"
deltaDataPath = workingDir + "/customer-data-delta/"

(spark.read
  .option("header", "true")
  .schema(inputSchema)
  .csv(inputPath) 
  .write
  .mode("overwrite")
  .format("delta")
  .partitionBy("Country")
  .save(deltaDataPath)  )

## Step 2: Time an Unoptimized Query by StockCode

Let's apply some of these optimizations to the `customer_data_delta` table.

Our data is partitioned by `Country`.

We want to query the data for `StockCode` equal to `22301`. 

We expect this query to be slow because we have to examine ALL OF the underlying data in `customer_data_delta` to find the desired `StockCode`. 

The data could be found on servers all over the world! That is, data can be coming from replications in other zones.

First, let's time the above query: you will need to form a DataFrame to pass to `preZorderQuery`.

 In Python we use the `timeit` 
<a href="https://docs.python.org/2/library/timeit.html" target="_blank">library function</a> 



Note that `timeit`  takes a function as input, so you need to define `myQuery` as a function.

In [34]:
# TODO
import timeit
deltaDataPath = workingDir + "/customer-data-delta/"

def myQuery():
  return """spark.sql("FILL_IN".format(deltaDataPath)).collect()"""

preTime = timeit.timeit(myQuery)

## Step 3: OPTIMIZE and ZORDER

Let's apply some of Databricks Delta's optimizations to `customer_data_delta`.

Our data is partitioned by `Country`.

Compact the files and re-order by `StockCode`.

In [36]:
# TODO
spark.sql("FILL_IN".format(deltaDataPath))

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 

 Py4JJavaError : An error occurred while calling o206.sql.
: org.apache.spark.sql.catalyst.parser.ParseException: 
mismatched input 'FILL_IN' expecting {'(', 'SELECT', 'FROM', 'ADD', 'DESC', 'WITH', 'VALUES', 'CREATE', 'TABLE', 'INSERT', 'DELETE', 'DESCRIBE', 'EXPLAIN', 'SHOW', 'USE', 'DROP', 'ALTER', 'MAP', 'SET', 'RESET', 'START', 'COMMIT', 'ROLLBACK', 'MERGE', 'UPDATE', 'CONVERT', 'REDUCE', 'REFRESH', 'CLEAR', 'CACHE', 'UNCACHE', 'DFS', 'TRUNCATE', 'ANALYZE', 'LIST', 'REVOKE', 'GRANT', 'LOCK', 'UNLOCK', 'MSCK', 'EXPORT', 'IMPORT', 'LOAD', 'OPTIMIZE', 'COPY'}(line 1, pos 0)

== SQL ==
FILL_IN
^^^

	at org.apache.spark.sql.catalyst.parser.ParseException.withCommand(ParseDriver.scala:241)
	at org.apache.spark.sql.catalyst.parser.AbstractSqlParser.parse(ParseDriver.scala:117)
	at org.apache.spark.sql.execution.SparkSqlParser.parse(SparkSqlParser.scala:55)
	at org.apache.spark.sql.catalyst.parser.AbstractSqlParser.parsePlan(ParseDriver.scala:69)
	at com.databricks.sql.parser.DatabricksSqlParser$$anonfun$parsePlan$1.apply(DatabricksSqlParser.scala:64)
	at com.databricks.sql.parser.DatabricksSqlParser$$anonfun$parsePlan$1.apply(DatabricksSqlParser.scala:61)
	at com.databricks.sql.parser.DatabricksSqlParser.parse(DatabricksSqlParser.scala:84)
	at com.databricks.sql.parser.DatabricksSqlParser.parsePlan(DatabricksSqlParser.scala:61)
	at org.apache.spark.sql.SparkSession$$anonfun$6.apply(SparkSession.scala:694)
	at org.apache.spark.sql.SparkSession$$anonfun$6.apply(SparkSession.scala:694)
	at org.apache.spark.sql.catalyst.QueryPlanningTracker.measurePhase(QueryPlanningTracker.scala:111)
	at org.apache.spark.sql.SparkSession.sql(SparkSession.scala:693)
	at sun.reflect.GeneratedMethodAccessor579.invoke(Unknown Source)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)


During handling of the above exception, another exception occurred:

 ParseException Traceback (most recent call last)
 <command-2761344356637390> in <module> 
 1 # TODO 
 ----> 2 spark . sql ( "FILL_IN" . format ( deltaDataPath ) ) 

 /databricks/spark/python/pyspark/sql/session.py in sql (self, sqlQuery) 
 834 [ Row ( f1 = 1 , f2 = u'row1' ) , Row ( f1 = 2 , f2 = u'row2' ) , Row ( f1 = 3 , f2 = u'row3' ) ] 
 835 """
 --> 836 return DataFrame ( self . _jsparkSession . sql ( sqlQuery ) , self . _wrapped ) 
 837 
 838 @ since ( 2.0 ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 71 raise AnalysisException ( s . split ( ': ' , 1 ) [ 1 ] , stackTrace ) 
 72 if s . startswith ( 'org.apache.spark.sql.catalyst.parser.ParseException: ' ) : 
 ---> 73 raise ParseException ( s . split ( ': ' , 1 ) [ 1 ] , stackTrace ) 
 74 if s . star

## Step 4: Time an Optimized Query by StockCode

Let's time the above query again: use the same methodology as for the pre-optimized query.

We expect `postTime` to be smaller than `preTime`.

Recall, you defined `myQuery` previously.

In [38]:
# TODO
postTime = timeit.timeit(FILL_IN)

In [39]:
# TEST  - Run this cell to test your solution.
print("Pre ZORDER time is {} s".format(preTime))
print("Post ZORDER time is {} s".format(postTime))

## Step 5: Apply VACUUM

Make sure you set the retention period to 0 to perform the operation immediately.

There should be only 1 file in each `Country` partition.

In [41]:
# TODO
spark.sql("FILL_IN".format(deltaDataPath))

In [42]:
# TEST - Run this cell to test your solution.
from functools import reduce
try:
  myList = filter(lambda p: "_delta_log" not in p.path, dbutils.fs.ls(deltaDataPath))  # Pick up list of subdirectories except for _delta_log
  myMap = map(lambda p: computeFileStats(p.path)[0], myList)                           # computeFileStats is a tuple of (numFiles, fileSize)
  numFilesOne = reduce(lambda a, b: a and b and 1, myMap)                              # AND every element with eachother and the value 1
except:
  numFilesOne = -99

dbTest("Delta-08-numFilesOne", 1, numFilesOne)

print("Tests passed!")

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Classroom-Cleanup<br>

Run the **`Classroom-Cleanup`** cell below to remove any artifacts created by this lesson.

In [44]:
%run "./Includes/Classroom-Cleanup"

## Summary
In this lesson, we showed you how to use Databricks Delta's advanced optimization strategies:
* ZORDER uses an algorithm to rewrite parquet files so that related data is co-located
* OPTIMIZE compacts small files into larger files of around 1GB (and solves the small file problem)
* VACUUM deletes the smaller files that were used to form the larger files by OPTIMIZE

## Review Questions

**Q:** Why are many small files problematic when doing queries on data backed by these?<br>
**A:** If there are many files, some of which might not be co-located, the principal sources of slowdown are
* network latency 
* (volume of) file metatadata 

**Q:** What do `OPTIMIZE` and `VACUUM` do?<br>
**A:** `OPTIMIZE` create larger files from a collection of smaller files and `VACUUM` deletes the invalid small files that were used in compaction.

**Q:** What size files does `OPTIMIZE` compact to and why that value?<br>
**A:** Small files are compacted to around 1GB; this value was determined by the Spark optimization team as a good compromise between speed and performace.

**Q:** What should one be careful of when using `VACUUM`?<br>
**A:** Don't set a retention interval shorter than seven days because old snapshots and uncommitted files can still be in use by concurrent readers or writers to the table.

**Q:** What does `ZORDER` do?<br>
**A:** It is a technique to colocate related information in the same set of files to improve query performance.

## Next Steps

Start the next lesson, [Architecture]($./Delta 07 - Architecture ).

## Additional Topics & Resources

* <a href="https://docs.databricks.com/delta/optimizations.html#" target="_blank">Optimizing Performance and Cost</a>
* <a href="http://parquet.apache.org/documentation/latest/" target="_blank">Parquet Metadata</a>
* <a href="https://en.wikipedia.org/wiki/Z-order_curve" target="_blank">Z-Order Curve</a>

-sandbox
&copy; 2020 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>